# Install and load all libraries

In [1]:
from ml_utils2 import *
from fastai.vision.all import *
import gc

# Creating differe dataloaders

In [2]:
images_df=pd.read_csv('data/images_df_final.csv', dtype={'image_id':str,'clip_name':str,'labels':str})

def fix_paths(x): return x.replace('data/train_images_c','data/train_images_crop')

images_df['image_id']=images_df['image_id'].map(fix_paths)

def clean_labels(c): return str([(i).strip().replace(" ", "_").replace("-", "_").replace("/", "_") for i in re.sub(r"[\[\]]",'',c).split(',')]).replace("'","").strip("'").strip("[").strip("]").replace(" ","").replace('monopolar_curved_scissors','monopolar_curved_scissor')
images_df['labels'] = images_df['labels'].apply(clean_labels)

def y1_labeller(i): return re.sub(r"[\[\]]",'',i).split(',')[0].strip()
def y2_labeller(i): return re.sub(r"[\[\]]",'',i).split(',')[1].strip()
def y3_labeller(i): return re.sub(r"[\[\]]",'',i).split(',')[2].strip()
def y4_labeller(i): return re.sub(r"[\[\]]",'',i).split(',')[3].strip()

images_df['y1_label'] = images_df.labels.map(y1_labeller)
images_df['y2_label'] = images_df.labels.map(y2_labeller)
images_df['y3_label'] = images_df.labels.map(y3_labeller)
images_df['y4_label'] = images_df.labels.map(y4_labeller)
images_df.sample(10)

,image_id,clip_name,labels,valid,y1_label,y2_label,y3_label,y4_label
2401822,data/train_images_crop/clip_019900/01785.jpg,clip_019900,"needle_driver,nan,needle_driver,prograsp_forceps",False,needle_driver,nan,needle_driver,prograsp_forceps
1435060,data/train_images_crop/clip_011889/01680.jpg,clip_011889,"needle_driver,nan,needle_driver,cadiere_forceps",False,needle_driver,nan,needle_driver,cadiere_forceps
916909,data/train_images_crop/clip_007594/01260.jpg,clip_007594,"prograsp_forceps,cadiere_forceps,nan,clip_applier",False,prograsp_forceps,cadiere_forceps,nan,clip_applier
2458389,data/train_images_crop/clip_020368/00285.jpg,clip_020368,"cadiere_forceps,bipolar_forceps,nan,monopolar_curved_scissor",False,cadiere_forceps,bipolar_forceps,nan,monopolar_curved_scissor
959049,data/train_images_crop/clip_007942/00585.jpg,clip_007942,"bipolar_forceps,nan,monopolar_curved_scissor,cadiere_forceps",False,bipolar_forceps,nan,monopolar_curved_scissor,cadiere_forceps
848068,data/train_images_crop/clip_007024/01080.jpg,clip_007024,"bipolar_forceps,nan,monopolar_curved_scissor,cadiere_forceps",False,bipolar_forceps,nan,monopolar_curved_scissor,cadiere_forceps
475795,data/train_images_crop/clip_003940/00090.jpg,clip_003940,"bipolar_forceps,nan,monopolar_curved_scissor,cadiere_forceps",False,bipolar_forceps,nan,monopolar_curved_scissor,cadiere_forceps
879808,data/train_images_crop/clip_007287/01305.jpg,clip_007287,"needle_driver,nan,needle_driver,cadiere_forceps",False,needle_driver,nan,needle_driver,cadiere_forceps
895145,data/train_images_crop/clip_007414/01635.jpg,clip_007414,"bipolar_forceps,nan,monopolar_curved_scissor,grasping_retractor",False,bipolar_forceps,nan,monopolar_curved_scissor,grasping_retractor
215606,data/train_images_crop/clip_001785/00975.jpg,clip_001785,"needle_driver,nan,needle_driver,cadiere_forceps",True,needle_driver,nan,needle_driver,cadiere_forceps


In [3]:
def get_dblock(item_tfms, batch_tfms):
    
    dblock = DataBlock(
        blocks=(ImageBlock,CategoryBlock(vocab=tools_list),CategoryBlock(vocab=tools_list),CategoryBlock(vocab=tools_list),CategoryBlock(vocab=tools_list)),
        n_inp=1,
        get_x=ColReader('image_id'),
        get_y=[ColReader('y1_label'),ColReader('y2_label'),ColReader('y3_label'),ColReader('y4_label')],
        splitter=splitter,
        item_tfms=item_tfms,
        batch_tfms=batch_tfms)
    
    return dblock


In [4]:
dblock = get_dblock(item_tfms=Resize((180,320), method='squish'), batch_tfms=aug_transforms(size=(180,320), min_scale=1))
dls = dblock.dataloaders(images_df, seed=42, n_workers=8)

In [5]:
def train(exp_name,arch,size,item_tfms,epochs=12):
    dblock = get_dblock(item_tfms=item_tfms, batch_tfms=[*aug_transforms(size=size, min_scale=1),Normalize.from_stats(*imagenet_stats)])
    dls = dblock.dataloaders(images_df, seed=42, n_workers=32)
    learn = vision_learner(dls, arch, loss_func=combo_loss, metrics=metrics_cfg, n_out=cfg(4)).to_fp16()
    learn.fine_tune(epochs, 0.01)
    learn.path=Path('models')
    learn.export(f'{arch}-model-ls.pkl')
    return learn.tta(dl=dls.valid)
 

In [6]:
res = (180,320)

In [7]:
archs = {
    'convnext_small_in22k': {
        (Resize(res, ResizeMethod.Squish), res),
    },
    # 'vit_small_patch16_224': {
    #     (Resize(res, ResizeMethod.Squish), 224),
    # },
    'swinv2_base_window12_192_22k':{
        (Resize(res, ResizeMethod.Squish), 192),
    },
    'swin_small_patch4_window7_224': {
        (Resize(res, ResizeMethod.Squish), 224)
    }
}

In [ ]:
tta_res = []

for arch, details in archs.items():
    for i, s in details:
        print('----',arch, ' --- ')
        print(s)
        print(i.name)
        tta_res.append(train('squish', arch, size=s, item_tfms=i))
        gc.collect()
        torch.cuda.empty_cache()


---- convnext_small_in22k  --- 
(180, 320)
Resize -- {'size': (320, 180), 'method': 'squish', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


epoch,train_loss,valid_loss,usm1_loss,usm2_loss,usm3_loss,usm4_loss,usm1_err,usm2_err,usm3_err,usm4_err,combo_err,time
0,2.846285,2.738796,0.676795,0.646919,0.696763,0.718318,0.038794,0.020474,0.030024,0.048174,0.137465,1:51:04


epoch,train_loss,valid_loss,usm1_loss,usm2_loss,usm3_loss,usm4_loss,usm1_err,usm2_err,usm3_err,usm4_err,combo_err,time


In [ ]:
for arch, details in archs.items():
    for i, s in details:
        print('----',arch, ' --- ')
        print(s)
        print(i.name)
        tta_res.append(train('squish', arch, size=s, item_tfms=i))
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
save_pickle('06_archs-small-ensemble-tta_res.pkl', tta_res)

In [ ]:
def decode_ensemble_result(output):
    output = first(zip(*output)) # unzipping preds and targes
    output+= output[1:3]        # increase weights of certain models with higher accuracy
    output = torch.stack(output)# stacking predctions from all models
    labels=[]
    df = pd.DataFrame(dls.valid.items)
    df.drop(['y1_label','y2_label','y3_label','y4_label','valid'], axis=1, inplace=True)
    for i in range(len(dls.c)):
        arm_preds = output[:,:,cfg(i):cfg(i+1)].mean(0);
        arm_idxs = arm_preds.argmax(dim=1)
        arm_vocab = np.array(dls.vocab[i])
        df[f'y{i+1}_label'] = arm_vocab[arm_idxs]
        
    df['labels_pred'] = '['+ df['y1_label']+', '+df['y2_label']+', '+df['y3_label']+', '+df['y4_label']+']'
    df['accurate_tf'] = (df['labels']==df['labels_pred'])
    df.drop(['y1_label','y2_label','y3_label','y4_label'], axis=1, inplace=True)
    return df


In [ ]:
output_df = decode_ensemble_result(tta_res)
output_df.to_csv('06_archs-small-ensemble-results.csv', index=False)
output_df['accurate_tf'].value_counts()/len(output_df)*100

In [ ]:
tta_prs = first(zip(*tta_res))

In [ ]:
tta_prs+= tta_prs[1:3]

In [ ]:
tta_prs_stacked = torch.stack(tta_prs)

In [ ]:
pr_usm1 = tta_prs_stacked[:,:,:cfg(1)].mean(0);
idxs1 = pr_usm1.argmax(dim=1)
vocab = np.array(dls.vocab[0])
usm1_lbl = vocab[idxs1]

pr_usm2 = tta_prs_stacked[:,:,cfg(1):cfg(2)].mean(0);
idxs2 = pr_usm2.argmax(dim=1)
vocab = np.array(dls.vocab[1])
usm2_lbl = vocab[idxs2]

pr_usm3 = tta_prs_stacked[:,:,cfg(2):cfg(3)].mean(0);
idxs3 = pr_usm3.argmax(dim=1)
vocab = np.array(dls.vocab[2])
usm3_lbl = vocab[idxs3]

pr_usm4 = tta_prs_stacked[:,:,cfg(3):cfg(4)].mean(0);
idxs4 = pr_usm4.argmax(dim=1)
vocab = np.array(dls.vocab[3])
usm4_lbl = vocab[idxs4]


In [ ]:
val_df = pd.DataFrame(dls.valid.items)
val_df.drop(['y1_label','y2_label','y3_label','y4_label','valid'], axis=1, inplace=True)
val_df['labels_pred'] = ['['+usm1_lbl[i]+','+usm2_lbl[i]+','+usm3_lbl[i]+','+usm4_lbl[i]+']' for i in range(len(usm1_lbl))]

val_df.head(100)

In [ ]:
pr_usm1 = tta_prs_stacked[:,:,:cfg(1)].mean(0);
idxs1 = pr_usm1.argmax(dim=1)
vocab = np.array(dls.vocab[0])
vocab[idxs1]

In [ ]:
tta_prs[0][:,cfg(1):cfg(2)].shape